In [1]:
import PyPDF2
import re
import random

# creating a pdf reader object
reader = PyPDF2.PdfReader("./Course_Schedule_2023-24-2.pdf")

# print the number of pages in pdf file
print(len(reader.pages))

entries = []
for page in reader.pages:
    entries.append(page.extract_text())

83


In [2]:
print(entries[5].replace("\n", ""))

 Course Schedule/ Classroom Allocation: 2023 -24/2 Semester                                                  [*This is a dynamic list. It will get updated as and when departments propose changes.]   Page 6 of 83 19-Jan-24 5:04:35 PM  S.No. Branch Course Name/Group Name  Slot Name Units(Credits) Course Type  Instructor  Instructor Email  Discussion Schedule  Tutorial Schedule  Practical Schedule singhappu@iitk.ac.in (O)  (BSBE SR) 15:00 -16:00  42 BSBE  BIOINFORMATICS & COMPUTATIONAL BIOLOGY(BSE322)  SLOT -4-4 3-0-0-0(9) DC / REGULAR  NITIN GUPTA(I)  guptan@iitk.ac.in (I) M (L01) Th (L01) 12:00 -13:15    43 BSBE  BIOINFORMATICS & COMPUTATIONAL BIOLOGY(BSE322A)  SLOT -4-4 3-0-0-1(10)  PRF / REGULAR  NITIN GUPTA(I)  guptan@iitk.ac.in (I) M (L01) Th (L01) 12:00 -13:15    44 BSBE  UG PROJECT (UGP -II)(BSE399A)  SLOT - Blank  0-0-2-0(9) UGP -2 / REGULAR  DUGCBSBE (I)  dugc_bsbe@iitk.ac.in (I)     45 BSBE  TISSUE ENGINEERING(BSE421A)  OE-1 3-0-0-2(11)  DE,PRF / REGULAR  DHIRENDRA S KATTI(I)  

In [4]:
number_match = re.compile(r'\n(\d+\s[A-Z].+?(?=\n\d+\s[A-Z]|\Z))', re.DOTALL)

In [5]:
courses_raw = []
for entry in entries:
    courses_raw.extend(number_match.findall(entry))

In [6]:
re_split_course_instructor = re.compile(r"\d+-\d+-\d+-\d+")
re_split_course_timing = re.compile(r"ac.in\s*\([IO]\),*\s+(?=[A-Z]|$)", re.DOTALL)
re_find_timings = re.compile(r"(.+?)(\d\d:\d\d)\s*-\s*(\d\d:\d\d)")
re_find_days = re.compile(r"(Th|M|T|W|F)")
re_find_course_details = re.compile(r"^(\d+)\s+([A-Z]+)\s+(.*?)\(([A-Z]+\d+[A-Z]*)\)")
re_split_professors = re.compile(r",\s+")
re_split_types = re.compile(r"([A-Za-z0-9]+)")

In [7]:
def remove_nested_parens(input_str):
    result = ""
    paren_level = 0
    for ch in input_str:
        if ch == "(":
            paren_level += 1
        elif (ch == ")") and paren_level:
            paren_level -= 1
        elif not paren_level:
            result += ch
    return result


def seperate_timetable(input_str: str):
    global err_count
    try:
        rest_time_split = re_split_course_timing.split(input_str)
        course, timing = rest_time_split[0], rest_time_split[1].strip()
        course += "iitk.ac.in"
    except IndexError as err:
        raise IndexError(f"Could not seperate timetable {rest_time_split}: {err}")
    timing_slots = re_find_timings.findall(timing)
    timings = []
    for t, s, e in timing_slots:
        t = remove_nested_parens(t)
        days = set(re_find_days.findall(t))
        for day in days:
            timings.append(f"{day} {s}-{e}")
    if len(timings) == 0:
        err_count -= 1
        raise ValueError(f"No timings found for {course}, skipping")
    return course, timings


def seperate_course_details(input_str: str):
    try:
        ci_split = re_split_course_instructor.split(input_str)
        course, instr = ci_split[0], ci_split[1]
    except IndexError as err:
        raise IndexError(f"Could not seperate course-instructor {input_str}: {err}")
    course_details = re_find_course_details.search(course).groups()
    assert len(course_details) == 4
    instr_split = re.split(r"\s\s+", instr)
    assert len(instr_split) >= 3
    credit = re.search(r"\((\d+)\)", instr_split[0]).groups()[0]
    credit = int(credit)
    return course_details, instr_split, credit

In [17]:
course = courses_raw[18]

In [18]:
course = course.replace("\n", "")
course

'19 AE SPACE DYNAMICS -I(AE641)  PG-5 3-0-0-0(9) DE / REGULAR  ASHISH TIWARI (I)  ashtew@iitk.ac.in (I) T (DJAC(206H)) 10:30 -11:45 ,Th (DJAC(206H)) 12:00 -13:15    '

In [19]:
crs, time = seperate_timetable(course)

In [20]:
crs, time

('19 AE SPACE DYNAMICS -I(AE641)  PG-5 3-0-0-0(9) DE / REGULAR  ASHISH TIWARI (I)  ashtew@iitk.iitk.ac.in',
 ['T 10:30-11:45', 'Th 12:00-13:15'])

In [21]:
try:
    ci_split = re_split_course_instructor.split(crs)
    course, instr = ci_split[0], ci_split[1]
except IndexError as err:
    raise IndexError(f"Could not seperate course-instructor {crs}: {err}")

In [22]:
course, instr

('19 AE SPACE DYNAMICS -I(AE641)  PG-5 ',
 '(9) DE / REGULAR  ASHISH TIWARI (I)  ashtew@iitk.iitk.ac.in')

In [25]:
course_details = re.search(
    r"^(\d+)\s+([A-Z]+)\s+(.*?)\(([A-Z]+\d+[A-Z]*)\)", 
    course).groups()
assert len(course_details) == 4
instr_split = re.split(r"\s\s+", instr)
assert len(instr_split) >= 3
credit = re.search(r"\((\d+)\)", instr_split[0]).groups()[0]
credit = int(credit)

In [26]:
course_details, instr_split, credit

(('19', 'AE', 'SPACE DYNAMICS -I', 'AE641'),
 ['(9) DE / REGULAR', 'ASHISH TIWARI (I)', 'ashtew@iitk.iitk.ac.in'],
 9)